# Expected Completion

In [16]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist #function to create adjacency matrix
import math
import tensorflow
import spektral
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from spektral.data import Dataset, BatchLoader, Graph, DisjointLoader
from spektral.transforms.normalize_adj import NormalizeAdj
from tensorflow.keras.layers import Dropout
from keras.callbacks import Callback, EarlyStopping
from spektral.layers import GCNConv, GlobalSumPool

In [17]:
plays = pd.read_csv("data/pass_receiver.csv")
pff = pd.read_csv("data/pffScoutingData.csv")
games = pd.read_csv("data/games.csv")

read in weekly tracking files and concatenate into one tracking file, then delete weeklies

In [18]:
week1 = pd.read_csv("data/week1.csv")
week2 = pd.read_csv("data/week2.csv")
week3 = pd.read_csv("data/week3.csv")
week4 = pd.read_csv("data/week4.csv")
week5 = pd.read_csv("data/week5.csv")
week6 = pd.read_csv("data/week6.csv")
week7 = pd.read_csv("data/week7.csv")
week8 = pd.read_csv("data/week8.csv")
tracking = pd.concat([week1,week2,week3,week4,week5,week6,week7,week8])


In [19]:
#save memory
del week1,week2,week3,week4,week5,week6,week7,week8

In [20]:
plays = plays.merge(games[["gameId","homeTeamAbbr"]], on = "gameId")
plays["possTeamScoreDiff"] = np.where(plays["homeTeamAbbr"]==plays["possessionTeam"],plays["preSnapHomeScore"]-plays["preSnapVisitorScore"],plays["preSnapVisitorScore"]-plays["preSnapHomeScore"])
plays["possTeamInLead"] = np.where(plays["possTeamScoreDiff"]>0,1,0)

,gameId,playId,playDescription,quarter,down,yardsToGo,possessionTeam,defensiveTeam,yardlineSide,yardlineNumber,...,pff_passCoverageType,gameId_playId,is_pass_failed,PassReceiver_nflId,PassReceiver_pff_role_num,PassReceiver_displayName,PassReceiver_short_name,homeTeamAbbr,possTeamScoreDiff,possTeamInLead
0,2021090900,97,(13:33) (Shotgun) T.Brady pass incomplete deep...,1,3,2,TB,DAL,TB,33,...,Man,2021090900_97,1,44896.0,Pass Route_#5,Chris Godwin,C.Godwin,TB,0,0
1,2021090900,137,(13:18) (Shotgun) D.Prescott pass deep left to...,1,1,10,DAL,TB,DAL,2,...,Zone,2021090900_137,0,42347.0,Pass Route_#1,Amari Cooper,A.Cooper,TB,0,0
2,2021090900,187,(12:23) (Shotgun) D.Prescott pass short middle...,1,2,6,DAL,TB,DAL,34,...,Zone,2021090900_187,0,46206.0,Pass Route_#4,Dalton Schultz,D.Schultz,TB,0,0
3,2021090900,282,(9:56) D.Prescott pass incomplete deep left to...,1,1,10,DAL,TB,TB,39,...,Zone,2021090900_282,1,52425.0,Pass Route_#3,CeeDee Lamb,C.Lamb,TB,0,0
4,2021090900,349,(9:46) (Shotgun) D.Prescott pass incomplete sh...,1,3,15,DAL,TB,TB,44,...,Zone,2021090900_349,1,52425.0,Pass Route_#4,CeeDee Lamb,C.Lamb,TB,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8563,2021110100,4310,(1:56) (Shotgun) P.Mahomes sacked at NYG 16 fo...,4,3,8,KC,NYG,NYG,8,...,Other,2021110100_4310,1,NaN,NaN,NaN,NaN,KC,0,0
8564,2021110100,4363,(1:07) (Shotgun) D.Jones pass short right to E...,4,1,10,NYG,KC,NYG,25,...,Zone,2021110100_4363,0,44835.0,Pass Route_#2,Evan Engram,E.Engram,KC,-3,0
8565,2021110100,4392,"(1:01) (No Huddle, Shotgun) D.Jones sacked at ...",4,2,7,NYG,KC,NYG,28,...,Zone,2021110100_4392,1,NaN,NaN,NaN,NaN,KC,-3,0
8566,2021110100,4411,"(:39) (No Huddle, Shotgun) D.Jones pass incomp...",4,3,15,NYG,KC,NYG,20,...,Zone,2021110100_4411,1,44835.0,Pass Route_#1,Evan Engram,E.Engram,KC,-3,0


add unique identifier of each play and frame (some playIds repeat over games)

In [21]:
tracking["comb_id"] = tracking["gameId"].astype("str")+" "+tracking["playId"].astype("str")
pff["comb_id"] = pff["gameId"].astype("str")+" "+pff["playId"].astype("str")
plays["comb_id"] = plays["gameId"].astype("str")+" "+plays["playId"].astype("str")
tracking["comb_and_frame"] = tracking["gameId"].astype("str")+" "+tracking["playId"].astype("str")+" "+tracking["frameId"].astype("str")

isolate frames at time of pass

In [22]:
#isolate just forward passes and leave out a certain frame (duplicate forward pass for a play)
just_forward_passes = tracking[(tracking["event"]=="autoevent_pass_forward") & (tracking["comb_and_frame"]!="2021091203 3740 32")& (tracking["team"]!="football")]
#bring in receiver info from plays
tracking_plays =just_forward_passes.merge(plays[["comb_id","possessionTeam","is_pass_failed","PassReceiver_nflId","down","yardsToGo","yardlineNumber","possTeamScoreDiff","possTeamInLead"]], how="left",on="comb_id" )
#and qb info from pff
tracking_merged_all = tracking_plays.merge(pff[["comb_id","nflId","pff_role"]], how = "left",on=["comb_id","nflId"])

,comb_id
39,2021090900 97
1020,2021090900 137
1867,2021090900 187
2590,2021090900 282
3574,2021090900 349
...,...
970115,2021110100 4049
971258,2021110100 4071
972242,2021110100 4113
974484,2021110100 4363


isolate tracking to just QB, OL, receiver and defense

In [27]:
#tracking_players_filtered = tracking_merged_all[(tracking_merged_all["nflId"]==tracking_merged_all["PassReceiver_nflId"])|(tracking_merged_all["pff_role"]=="Pass")|(tracking_merged_all["pff_role"]=="Pass Block")|(tracking_merged_all["team"]!=tracking_merged_all["possessionTeam"])]
tracking_players_filtered = tracking_merged_all[(tracking_merged_all["nflId"]==tracking_merged_all["PassReceiver_nflId"])|(tracking_merged_all["pff_role"]=="Pass")|(tracking_merged_all["team"]!=tracking_merged_all["possessionTeam"])]

# Function to take in 1 pass play and put it in graph format( list containing adjacency matrix, features, y) this will then be parallelized for all passes in the data

In [28]:
#categories = ["Coverage","Pass","Pass Block","Pass Rush", "Pass Route"]
categories = ["Coverage","Pass","Pass Rush", "Pass Route"]

In [29]:
def angle_relative_diff(a,b):
    diff_list = []
    for i in range(len(a)):
        difference = a[i] - b[0]
        if difference < -180:
            difference +=360
        elif difference > 180:
            difference -=360
        diff_list.append(difference)
    return diff_list

In [33]:
def one_play_graph_function(data):
    ##one_play_formatted_list = []
    data = data.reset_index(drop=True)
    #get y, the response variable for this play (binary pass completed or not) and possesionteam (for CV)
    y = data["is_pass_failed"][0]
    team = data["possessionTeam"][0]
    #put play in adjacency matrix format, the distance of all players to each other (this will then be filtered to just link relevant players to QB or WR)
    data_dist = data[["x","y"]]
    adj = cdist(data_dist.values, data_dist.values)
    for index,row in data.iterrows():
        #connect QB to pass rush/block and wr, WR to coverage and QB, remove other connections by setting to zero
        if (row["pff_role"]=="Coverage" )| (row["pff_role"]=="Pass Route"):
            condition = ((data["pff_role"]=="Pass Block")|(data["pff_role"]=="Pass Rush"))
        else:
            condition = ((data["pff_role"]=="Coverage"))
        adj[index][condition] = 0
    #then put play in features format, getting a table containing position, speed, orientation etc
    data["onPossessionTeam"] = np.where(data["possessionTeam"]==data["team"],1,0)
    QB= data[data["pff_role"]=="Pass"]
    data["rel_o"] = np.where(data["pff_role"]=="Pass Route", data["o"]-QB["o"].iloc[0],0)
    data["rel_dir"] = np.where(data["pff_role"]=="Pass Route",data["dir"]-QB["dir"].iloc[0],0)
    data 
    data = data[["onPossessionTeam","frameId","s","rel_dir","rel_o","pff_role","down","yardsToGo","yardlineNumber","possTeamScoreDiff","possTeamInLead"]][data["pff_role"]=="Pass Route"]
    #one_hot = pd.get_dummies(data['pff_role'].astype(pd.CategoricalDtype(categories=categories)))
    #data= data.drop('pff_role',axis = 1)
    # Join the encoded df
    data = data.join(one_hot)
    one_play_formatted_list = {"team":team,"y":y,"graph_a":adj,"graph_x":data}
    return one_play_formatted_list

In [34]:
graph_list = []
for i in range(len(tracking_players_filtered.comb_id.unique())):
#for i in range(10):
    
    one_play = tracking_players_filtered[tracking_players_filtered.comb_id==tracking_players_filtered.comb_id.unique()[i]]
    #print(one_game.comb_id.unique()[i])
    one_play = one_play.reset_index(drop=True)
    result = one_play_graph_function(one_play)
    graph_list.append(result.copy())


get list of indexes of possession teams to allow masking for Cross Validation

In [35]:
team_list = []
for i in range(len(graph_list)):
    team_list.append(graph_list[i]["team"])
unique_team_list = list(set(team_list))

# GNN Functions

In [36]:
#variables
learning_rate = 1e-3  # Learning rate
epochs = 100  # Number of training epochs
es_patience = 10  # Patience for early stopping
batch_size = 21  # Batch size
callbacks = [EarlyStopping(monitor='val_loss', patience=10, verbose=0)]

put data in graph format to feed in to GNN

In [38]:
class GraphDataset(Dataset):
    def __init__(self, n_samples, df, n_outcome=1, **kwargs):
        self.n_samples = n_samples
        self.df = df  
        self.n_outcome = n_outcome
        super().__init__(**kwargs)

    def read(self):
        output = []
        for i in range(self.n_samples):
            # Node features
            iter_x = self.df[i]["graph_x"].copy()
            #print(iter_x)
            x = np.array(iter_x)#.reshape(len(iter_x),9)

            # Edges
            iter_a =  self.df[i]["graph_a"].copy()
            the_length = len(iter_a)
            a = np.array(iter_a).reshape(the_length,the_length)
            #print(iter_a)
            y = int(self.df[i]["y"])
            
           
            output.append(Graph(x=x, a=a, y=y))
        return(output)

train xCompletion model

In [39]:
class xCompletion(Model):

    def __init__(self, n_hidden, n_labels):
        super().__init__()
        self.graph_conv = GCNConv(n_hidden)
        self.pool = GlobalSumPool()
        self.dropout = Dropout(0.5)
        self.dense = Dense(n_labels, 'sigmoid')

    def call(self, inputs):
        out = self.graph_conv(inputs)
        out = self.dropout(out)
        out = self.pool(out)
        out = self.dense(out)

        return out

Cross Validation with k = 32 by loop through and for each team use as val (remove from training) and store predictions

In [40]:
prediction_list = []
for i in range(len(unique_team_list)):
    team_in = [j for j, e in enumerate(team_list) if e == unique_team_list[i]]
    team_out = [j for j, e in enumerate(team_list) if e != unique_team_list[i]]
    training_data = [graph_list[i] for i in team_out]
    val_data = [graph_list[i] for i in team_in]
    all_data = graph_list
    len_train = len(training_data)
    len_val = len(val_data)
    len_all = len(graph_list)
    data_tr = GraphDataset(n_samples = len_train, df = training_data, transforms=NormalizeAdj())
    data_va = GraphDataset(n_samples = len_val, df = val_data, transforms=NormalizeAdj())
    data_all = GraphDataset(n_samples = len_all, df=graph_list, transforms=NormalizeAdj())

# Data loaders
    loader_tr = BatchLoader(data_tr, batch_size=batch_size, epochs=epochs)
    loader_va = BatchLoader(data_va, batch_size=batch_size)
    loader_all = BatchLoader(data_all, batch_size=batch_size)

    model = xCompletion(200, 1)
    model.compile('adam', "binary_crossentropy","binary_accuracy")

    model.fit(loader_tr.load(), validation_data= loader_va.load(), steps_per_epoch=loader_tr.steps_per_epoch,
    validation_steps=loader_va.steps_per_epoch, epochs=100,
      callbacks=callbacks)
    predictions = model.predict(loader_all.load(), steps =loader_all.steps_per_epoch)
    prediction_list.append(predictions)
    

Epoch 1/100
348/348 [==============================] - 3s 4ms/step - loss: 14.3958 - binary_accuracy: 0.5445 - val_loss: 7.2850 - val_binary_accuracy: 0.5397
Epoch 2/100
348/348 [==============================] - 1s 3ms/step - loss: 11.1217 - binary_accuracy: 0.5339 - val_loss: 4.1246 - val_binary_accuracy: 0.5159
Epoch 3/100
348/348 [==============================] - 1s 3ms/step - loss: 8.1993 - binary_accuracy: 0.5324 - val_loss: 5.3841 - val_binary_accuracy: 0.5476
Epoch 4/100
348/348 [==============================] - 1s 3ms/step - loss: 6.1777 - binary_accuracy: 0.5446 - val_loss: 1.8830 - val_binary_accuracy: 0.5794
Epoch 5/100
348/348 [==============================] - 1s 3ms/step - loss: 4.3923 - binary_accuracy: 0.5476 - val_loss: 2.1728 - val_binary_accuracy: 0.5476
Epoch 6/100
348/348 [==============================] - 1s 3ms/step - loss: 3.2520 - binary_accuracy: 0.5468 - val_loss: 1.3559 - val_binary_accuracy: 0.4841
Epoch 7/100
348/348 [==============================] - 1

In [53]:
model_output = np.mean(prediction_list, axis=0)

In [116]:
#pass_forwards = tracking[["comb_id"]][(tracking["event"]=="autoevent_pass_forward")|(tracking["event"]=="pass_forward")].drop_duplicates()#.groupby("comb_id").min()
play_with_pass = plays[plays["comb_id"].isin(tracking_players_filtered.comb_id)]
play_with_pass = play_with_pass.drop_duplicates(subset="comb_id")
play_with_pass["pred"] = model_output.tolist()
play_with_pass.to_csv("prediction_output.csv")